In [ ]:
from selenium import webdriver
import time

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument("--incognito")

driver = webdriver.Chrome(executable_path='/Users/ruilinzhao/Downloads/chromedriver', chrome_options=option)

categories = ["zoo", "hiking", "aquarium", "waterfall", "beach", "museum", "mountain", "ski resort", "amusement park"]
cat_http = []
for category in categories:
    cat_http.append(category.replace(' ', '%20'))

with open("attractions.txt", 'w') as file:
    for category in cat_http:
        for page in range(1,3):
            driver.get("https://www.tripadvisor.com/Search?redirect&uiOrigin=MASTHEAD&ssrc=A&pid=3826&searchSessionId=61EDF61512048A545712B0BB4AF20AC61554766645318ssid&supportedSearchTypes=find_near_stand_alone_query&geo=191&q="+ category +"&enableNearPage=true&social_typeahead_2018_feature=true")
            time.sleep(3)

            if page == 1:
                destinations = driver.find_elements_by_xpath("//div[@class='location-meta-block']//div")
            else:
                button = driver.find_element_by_xpath("//div[@class='pageNumbers']//a[@data-page=" + str(page) + "]")
                button.click()
                time.sleep(1)
                destinations = driver.find_elements_by_xpath("//div[@class='location-meta-block']//div")
                
            htmls = []
            for i in destinations[::6]:
                htmls.append(i.get_attribute("onclick").split("'/")[1].split("',")[0])
    
            for html in htmls:
                driver.get("https://www.tripadvisor.com/" + html)
                time.sleep(1)
                attraction = driver.find_element_by_xpath("//h1[@id='HEADING']").text
                rating = driver.find_element_by_xpath("//div[@class='section rating']//span[@class='overallRating']").text
                intro = driver.find_element_by_xpath("//div[@class='attractions-attraction-detail-about-card-AttractionDetailAboutCard__section--1_Efg']").text.replace('\n', ' ')
                address = driver.find_element_by_xpath("//span[@class='textAlignWrapper address']").text
                img = driver.find_element_by_xpath("//div[@class='prw_rup prw_common_basic_image xlarge landscape']//img").get_attribute("src")
                file.write(category.replace('%20', ' ') + '\t' + attraction + '\t' + address + '\t' + rating + '\t' + intro + '\t' + img + '\n')


In [ ]:
with open("attractions.txt", 'r') as file:
    content = file.read()
    addresses = content.split("\t")[2:][::5]
    cities = []
    for address in addresses:
        if (len(address.split(", "))) > 1:
            cities.append(address.split(", ")[-2])

cities = list(set(cities))

In [ ]:
from selenium.webdriver.common.keys import Keys

option = webdriver.ChromeOptions()
option.add_argument("--incognito")
driver = webdriver.Chrome(executable_path='/Users/ruilinzhao/Downloads/chromedriver', chrome_options=option)


with open("hotels.txt", 'w') as file:
    for city in cities:
        driver.get("https://www.booking.com/searchresults.en-us.html?label=gen173nr-1FCAEoggI46AdIM1gEaKsCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuAKO_L3lBcACAQ&sid=6c0e845bab923f385ebb787605fee81c&sb=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.html%3Flabel%3Dgen173nr-1FCAEoggI46AdIM1gEaKsCiAEBmAExuAEHyAEM2AEB6AEB-AECiAIBqAIDuAKO_L3lBcACAQ%3Bsid%3D6c0e845bab923f385ebb787605fee81c%3Bsb_price_type%3Dtotal%26%3B&ss="+city.replace(' ', '+')+"&checkin_year=2019&checkin_month=5&checkin_monthday=30&checkout_year=2019&checkout_month=5&checkout_monthday=31")
        time.sleep(2)

        try:
            driver.find_element_by_xpath("//a[@data-id='oos-1']//span").click()
        except:
            pass
        time.sleep(2)

        names = driver.find_elements_by_xpath("//span[@class='sr-hotel__name\n']")
        ratings = driver.find_elements_by_xpath("//div[@class='bui-review-score__badge']")
        prices = driver.find_elements_by_xpath("//b")
        imgs = driver.find_elements_by_xpath("//img[@class='hotel_image']")
        htmls = driver.find_elements_by_xpath("//a[@class='hotel_name_link url']")
        for i in range(min(len(names),len(ratings), len(prices), len(imgs), len(htmls), 10)):
            file.write(names[i].text + "\t" + city + "\t" + ratings[i].text + "\t" + prices[i].text + "\t" + imgs[i].get_attribute("src") + "\t" + htmls[i].get_attribute("href") +"\n")
            


In [ ]:
driver = webdriver.Chrome(executable_path='/Users/ruilinzhao/Downloads/chromedriver', chrome_options=option)
with open("restaurants.txt", 'w') as file:
    for city in cities:
        driver.get("https://www.yelp.com/search?find_desc=Restaurants&find_loc=" + city.replace(' ', '+'))
        time.sleep(2)

        names = driver.find_elements_by_xpath("//div[@class='lemon--div__373c0__1mboc businessName__373c0__1fTgn border-color--default__373c0__2oFDT']")
        ratings = driver.find_elements_by_xpath("//div[@class='lemon--div__373c0__1mboc attribute__373c0__1hPI_ display--inline-block__373c0__2de_K u-space-r1 border-color--default__373c0__2oFDT']//div")
        imgs = driver.find_elements_by_xpath("//img[@class='lemon--img__373c0__3GQUb photo-box-img__373c0__O0tbt']")
        categories = driver.find_elements_by_xpath("//div[@class='lemon--div__373c0__1mboc priceCategory__373c0__3zW0R border-color--default__373c0__2oFDT']")
        addresses = driver.find_elements_by_xpath("//address[@class='domtags--address__373c0__cgebO']")
        htmls = driver.find_elements_by_xpath("//div[@class='lemon--div__373c0__1mboc businessName__373c0__1fTgn border-color--default__373c0__2oFDT']//a")
        for i in range(3,13):
            try:
                file.write(names[i].text.split('. ')[1] + '\t' + city + '\t' + addresses[i].text + '\t' + ratings[i].get_attribute("aria-label").split(' ')[0] + '\t' + str(categories[i].text.count("$")) + '\t' + categories[i].text[categories[i].text.count("$"):] + '\t' + imgs[i].get_attribute("src") + '\t' + htmls[i].get_attribute("href") +'\n')
            except:
                pass
            